<a href="https://colab.research.google.com/github/qq2100803/CryptoMamba/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Cloning
!git clone https://github.com/MShahabSepehri/CryptoMamba.git
%cd CryptoMamba/

fatal: destination path 'CryptoMamba' already exists and is not an empty directory.
/content/CryptoMamba


In [ ]:
# @title Installing requirements

%%capture


In [ ]:
# @title Prediction
!python scripts/one_day_pred.py --config cmamba_v --ckpt_path ./checkpoints/cmamba_v.ckpt --date 2024-12-15

['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']

Prediction date: 2024-12-15
Prediction: 102174.26
Today value: 101372.97
Smart trade: 39.21% buy
Vanilla trade: -


In [ ]:
# @title Training
!python scripts/training.py --config cmamba_v

In [1]:
#@title 配置变量  { display-mode: "form" }
REPO = "https://github.com/MShahabSepehri/CryptoMamba.git"
WORKDIR = "/content/CryptoMamba_run"
DATA_DIR = WORKDIR + "/data"
CHECKPOINTS_DIR = WORKDIR + "/checkpoints"
RESULTS_DIR = WORKDIR + "/results"
CONFIG = "cmamba_v"  # 可改为需要的 config 名称
PYTHON_BIN = "python3"  # Colab 使用系统 python
WINDOW_DAYS = 365
TEST_WINDOW_DAYS = 30
ROLL_STEP_DAYS = 30
EPOCHS_FOR_QUICK_RUN = 3  # 用于快速验证，可调整为生产值


In [4]:
# 在 Colab 中执行
%%bash
set -euo pipefail
WORKDIR="/content/CryptoMamba_run"
REPO="https://github.com/MShahabSepehri/CryptoMamba.git"
rm -rf "$WORKDIR"
mkdir -p "$WORKDIR"
git clone --depth 1 "$REPO" "$WORKDIR/repo"
cd "$WORKDIR/repo"

# 使用 Colab 的 Python 环境安装 requirements
pip install --upgrade pip
if [ -f requirements.txt ]; then
pip install mamba-ssm[causal-conv1d] --no-build-isolation
pip install -r requirements.txt

else
  echo "未找到 requirements.txt，跳过依赖安装"
fi


  Using cached mamba_ssm-2.2.5.tar.gz (113 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.5-cp312-cp312-linux_x86_64.whl size=532566033 sha256=c8b65fcabfb49a94456c9971619007218e4073f19a84fb6b3894f33d43bee4a1
  Stored in directory: /root/.cache/pip/wheels/21/55/c4/85b634055d6a9b599d27f5cbeacf353c6c532d8e2d8769960b
  Created wheel for causal-conv1d: filename=causal_conv1d-1.5.2-cp312-cp312-linux_x86_64.whl size=151160839 sha256=7cc4ae241543f93acfa4d96ecbc43e532f3aeed171f88c859f873f1a617e9606
  Stored in directory: /root/.cache/pip/wheels/b4/a5/a7/8d0ecdd7a890f6339

Cloning into '/content/CryptoMamba_run/repo'...


In [7]:
# 拉取 BTC-USD 日线并保存为 repo/data/raw.csv（列名 Timestamp,Open,High,Low,Close,Volume）
import yfinance as yf
import pandas as pd
import os

WORKDIR = "/content/CryptoMamba_run"
OUT_DIR = os.path.join(WORKDIR, "data")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_CSV = os.path.join(OUT_DIR, "btc_usd_daily.csv")
REPO_RAW = os.path.join(WORKDIR, "repo", "data", "raw.csv")
os.makedirs(os.path.dirname(REPO_RAW), exist_ok=True)

# 安装 yfinance（若尚未安装）
try:
    import yfinance
except Exception:
    import sys
    !pip install yfinance --quiet

# 下载数据
df = yf.download("BTC-USD", start="2010-01-01", progress=False)
if df.empty:
    raise SystemExit("yfinance 未返回数据，检查网络或重试")

# 重整列并导出
df = df.reset_index()
df = df.rename(columns={"Date":"Timestamp"})
df["Timestamp"] = df["Timestamp"].dt.strftime("%Y-%m-%d")
df = df[["Timestamp","Open","High","Low","Close","Volume"]]
df.to_csv(OUT_CSV, index=False)
df.to_csv(REPO_RAW, index=False)
print("保存完成:", OUT_CSV)
print("repo path:", REPO_RAW)
print(df.tail(3))


/tmp/ipython-input-2837690587.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("BTC-USD", start="2010-01-01", progress=False)


保存完成: /content/CryptoMamba_run/data/btc_usd_daily.csv
repo path: /content/CryptoMamba_run/repo/data/raw.csv
Price    Timestamp           Open           High            Low  \
Ticker                    BTC-USD        BTC-USD        BTC-USD   
4031    2025-09-30  114396.523438  114836.617188  112740.562500   
4032    2025-10-01  114057.593750  118648.929688  113981.398438   
4033    2025-10-03  120606.320312  120606.320312  119965.937500   

Price           Close       Volume  
Ticker        BTC-USD      BTC-USD  
4031    114056.085938  58986330258  
4032    118648.929688  71328680132  
4033    120403.328125  65915445248  


In [18]:
# 在 Colab Python 单元中执行
import pandas as pd, os, json
WORKDIR = "/content/CryptoMamba_run"
os.chdir(os.path.join(WORKDIR, "repo"))
df = pd.read_csv("data/raw.csv")
# 兼容不同 CSV 中的日期列名
date_col = None
for candidate in ["Date","date","timestamp","Timestamp"]:
    if candidate in df.columns:
        date_col = candidate
        break
if date_col is None:
    raise SystemExit("未找到日期列，请检查 data/raw.csv 的列名（需要 Date 或类似列）")
df['Date'] = pd.to_datetime(df[date_col], errors='coerce')
df = df.dropna(subset=['Date']).sort_values('Date').reset_index(drop=True)
dates = df['Date'].dt.date.tolist()

WINDOW = int(os.environ.get('WINDOW_DAYS', str(WINDOW_DAYS)))
TEST = int(os.environ.get('TEST_WINDOW_DAYS', str(TEST_WINDOW_DAYS)))
STEP = int(os.environ.get('ROLL_STEP_DAYS', str(ROLL_STEP_DAYS)))

tasks = []
start_idx = 0
while start_idx + WINDOW + TEST <= len(dates):
    train_start = dates[start_idx]
    train_end = dates[start_idx + WINDOW - 1]
    test_start = dates[start_idx + WINDOW]
    test_end = dates[start_idx + WINDOW + TEST - 1]
    tasks.append((str(train_start), str(train_end), str(test_start), str(test_end)))
    start_idx += STEP

with open("rolling_tasks.json", "w") as f:
    json.dump(tasks, f, indent=2)

print(f"生成 {len(tasks)} 个滚动任务，保存在 rolling_tasks.json")


生成 122 个滚动任务，保存在 rolling_tasks.json


In [17]:
%%bash
set -euo pipefail
cd /content/CryptoMamba_run/repo
pip install ruamel.yaml --quiet
WORKDIR="/content/CryptoMamba_run"
CHECKPOINTS_DIR="$WORKDIR/checkpoints"
RESULTS_DIR="$WORKDIR/results"
mkdir -p "$CHECKPOINTS_DIR" "$RESULTS_DIR"
CHECKPOINTS_ROOT="$WORKDIR/repo/CryptoMamba/logs/CMamba"
CONFIG="cmamba_v"
PYTHON_BIN="python3"
EPOCHS_FOR_QUICK_RUN=3

TASKS_COUNT=$(python3 -c "import json,sys;print(len(json.load(open('rolling_tasks.json'))))")
if [ "$TASKS_COUNT" -eq 0 ]; then
  echo "任务数为 0，请检查数据长度和窗口参数。" >&2
  exit 1
fi

TASK_IDX=1
for TASK in $(jq -c '.[]' rolling_tasks.json); do
  TASK_IDX=$((TASK_IDX+1))
  TRAIN_START=$(echo "$TASK" | jq -r '.[0]')
  TRAIN_END=$(echo "$TASK" | jq -r '.[1]')
  TEST_START=$(echo "$TASK" | jq -r '.[2]')
  TEST_END=$(echo "$TASK" | jq -r '.[3]')

  echo "任务 $TASK_IDX / $TASKS_COUNT: 训练 $TRAIN_START -> $TRAIN_END 测试 $TEST_START -> $TEST_END"

  TMP_CONFIG="tmp_config_${TASK_IDX}.yaml"
  if [ -f "configs/training/${CONFIG}.yaml" ]; then
    cp "configs/training/${CONFIG}.yaml" "$TMP_CONFIG"
  elif [ -f "configs/${CONFIG}.yaml" ]; then
    cp "configs/${CONFIG}.yaml" "$TMP_CONFIG"
  else
    echo "未找到源配置文件 configs/training/${CONFIG}.yaml 或 configs/${CONFIG}.yaml，跳过该任务" >&2
    continue
  fi

  # 使用 Python 修改 YAML，避免依赖 yq
  python3 - <<PY
import ruamel.yaml, sys, os
fn="$TMP_CONFIG.yaml"
y=ruamel.yaml.YAML()
with open(fn) as f:
    cfg=y.load(f) or {}
cfg.setdefault('data',{})['start_date']="$TRAIN_START"
cfg['data']['end_date']="$TRAIN_END"
cfg['data']['test_start']="$TEST_START"
cfg['data']['test_end']="$TEST_END"
cfg.setdefault('training',{})['epochs']=${EPOCHS_FOR_QUICK_RUN}
with open(fn,'w') as f:
    y.dump(cfg,f)
print("已写入临时配置",fn)
PY

  echo "训练模型..."
  # 若脚本接受 --save_dir 或自动保存到 checkpoints，按仓库实际参数修改
  "$PYTHON_BIN" scripts/training.py --config "$TMP_CONFIG" --logdir "$CHECKPOINTS_ROOT" --save_checkpoints --max_epochs ${EPOCHS_FOR_QUICK_RUN} 2>&1 | tee "$RESULTS_DIR/train_${TASK_IDX}.log" || true

  # 在 logs/CMamba 下查找最新的 .ckpt（跨 version 目录）
  LATEST_CKPT=$(find "$CHECKPOINTS_ROOT" -type f -name "*.ckpt" -print0 2>/dev/null | xargs -0 ls -t 2>/dev/null | head -n1 || true)
  if [ -z "$LATEST_CKPT" ]; then
    echo "未找到 checkpoint（在 $CHECKPOINTS_ROOT 下没有 .ckpt 文件）。请查看 $RESULTS_DIR/train_${TASK_IDX}.log 以排查训练保存路径。" >&2
    continue
  fi
  echo "找到最新 checkpoint: $LATEST_CKPT"

  echo "评估模型..."
  $PYTHON_BIN scripts/evaluation.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" > "$RESULTS_DIR/eval_${TASK_IDX}.log" || true

  echo "回测（test split）..."
  $PYTHON_BIN scripts/simulate_trade.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" --split test --trade_mode smart > "$RESULTS_DIR/backtest_${TASK_IDX}.log" || true

  echo "准备单日预测输入 one_day_input.csv..."
  $PYTHON_BIN - <<PY
import pandas as pd
df=pd.read_csv("data/raw.csv")
df['Date']=pd.to_datetime(df['Date'])
end=pd.to_datetime("$TEST_END")
subset=df[df['Date']<=end].tail(90)
subset.to_csv("one_day_input.csv",index=False)
print("one_day_input.csv 已生成")
PY

  echo "单日预测..."
  $PYTHON_BIN scripts/one_day_pred.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" --input_csv one_day_input.csv > "$RESULTS_DIR/one_day_pred_${TASK_IDX}.log" || true

  echo "任务 $TASK_IDX 完成，结果保存在 $RESULTS_DIR"
done

echo "全部任务完成。汇总评估日志在 $RESULTS_DIR"


任务 2 / 122: 训练 2014-09-17 -> 2015-09-16 测试 2015-09-17 -> 2015-10-16
已写入临时配置 tmp_config_2.yaml
训练模型...
Seed set to 23
Traceback (most recent call last):
  File "/content/CryptoMamba_run/repo/scripts/training.py", line 136, in <module>
    config = io_tools.load_config_from_yaml(f'{ROOT}/configs/training/{args.config}.yaml')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CryptoMamba_run/repo/utils/io_tools.py", line 38, in load_config_from_yaml
    raise ValueError(f'Config file ({path}) does not exist.')
ValueError: Config file (/content/CryptoMamba_run/repo/configs/training/tmp_config_2.yaml.yaml) does not exist.
找到最新 checkpoint: tmp_config_2.yaml
评估模型...
回测（test split）...
准备单日预测输入 one_day_input.csv...


Seed set to 23
Traceback (most recent call last):
  File "/content/CryptoMamba_run/repo/scripts/evaluation.py", line 176, in <module>
    config = io_tools.load_config_from_yaml(f'{ROOT}/configs/training/{args.config}.yaml')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CryptoMamba_run/repo/utils/io_tools.py", line 38, in load_config_from_yaml
    raise ValueError(f'Config file ({path}) does not exist.')
ValueError: Config file (/content/CryptoMamba_run/repo/configs/training/tmp_config_2.yaml.yaml) does not exist.
Traceback (most recent call last):
  File "/content/CryptoMamba_run/repo/scripts/simulate_trade.py", line 195, in <module>
    config = io_tools.load_config_from_yaml(f'{ROOT}/configs/training/{conf}.yaml')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/CryptoMamba_run/repo/utils/io_tools.py", line 38, in load_config_from_yaml
    raise ValueError(f'Config file 

CalledProcessError: Command 'b'set -euo pipefail\ncd /content/CryptoMamba_run/repo\npip install ruamel.yaml --quiet\nWORKDIR="/content/CryptoMamba_run"\nCHECKPOINTS_DIR="$WORKDIR/checkpoints"\nRESULTS_DIR="$WORKDIR/results"\nmkdir -p "$CHECKPOINTS_DIR" "$RESULTS_DIR"\nCHECKPOINTS_ROOT="$WORKDIR/repo/CryptoMamba/logs/CMamba"\nCONFIG="cmamba_v"\nPYTHON_BIN="python3"\nEPOCHS_FOR_QUICK_RUN=3\n\nTASKS_COUNT=$(python3 -c "import json,sys;print(len(json.load(open(\'rolling_tasks.json\'))))")\nif [ "$TASKS_COUNT" -eq 0 ]; then\n  echo "\xe4\xbb\xbb\xe5\x8a\xa1\xe6\x95\xb0\xe4\xb8\xba 0\xef\xbc\x8c\xe8\xaf\xb7\xe6\xa3\x80\xe6\x9f\xa5\xe6\x95\xb0\xe6\x8d\xae\xe9\x95\xbf\xe5\xba\xa6\xe5\x92\x8c\xe7\xaa\x97\xe5\x8f\xa3\xe5\x8f\x82\xe6\x95\xb0\xe3\x80\x82" >&2\n  exit 1\nfi\n\nTASK_IDX=1\nfor TASK in $(jq -c \'.[]\' rolling_tasks.json); do\n  TASK_IDX=$((TASK_IDX+1))\n  TRAIN_START=$(echo "$TASK" | jq -r \'.[0]\')\n  TRAIN_END=$(echo "$TASK" | jq -r \'.[1]\')\n  TEST_START=$(echo "$TASK" | jq -r \'

In [15]:
%%bash
set -euo pipefail
cd /content/CryptoMamba_run/repo

# 安装必要库用于 yaml 修改（若已安装会跳过）
pip install ruamel.yaml --quiet

WORKDIR="/content/CryptoMamba_run"
# 指向仓库日志目录（含 version_x/checkpoints 子目录）
CHECKPOINTS_ROOT="$WORKDIR/repo/CryptoMamba/logs/CMamba"
RESULTS_DIR="$WORKDIR/results"
mkdir -p "$RESULTS_DIR"

CONFIG="cmamba_v"
PYTHON_BIN="python3"
EPOCHS_FOR_QUICK_RUN=3

# 检查 rolling_tasks.json
TASKS_COUNT=$(python3 -c "import json,sys;print(len(json.load(open('rolling_tasks.json'))))")
if [ "$TASKS_COUNT" -eq 0 ]; then
  echo "任务数为 0，请检查 rolling_tasks.json 和数据长度。" >&2
  exit 1
fi

TASK_IDX=0
#for TASK in $(jq -c '.[]' rolling_tasks.json); do
  TASK_IDX=$((TASK_IDX+1))
  TRAIN_START=$(echo "$TASK" | jq -r '.[0]')
  TRAIN_END=$(echo "$TASK" | jq -r '.[1]')
  TEST_START=$(echo "$TASK" | jq -r '.[2]')
  TEST_END=$(echo "$TASK" | jq -r '.[3]')

  echo "任务 $TASK_IDX / $TASKS_COUNT: 训练 $TRAIN_START -> $TRAIN_END 测试 $TEST_START -> $TEST_END"

  TMP_CONFIG="tmp_config_${TASK_IDX}.yaml"
  if [ -f "configs/training/${CONFIG}.yaml" ]; then
    cp "configs/training/${CONFIG}.yaml" "$TMP_CONFIG"
  elif [ -f "configs/${CONFIG}.yaml" ]; then
    cp "configs/${CONFIG}.yaml" "$TMP_CONFIG"
  else
    echo "未找到源配置文件 configs/training/${CONFIG}.yaml 或 configs/${CONFIG}.yaml，跳过该任务" >&2
    continue
  fi

  # 修改临时配置
  python3 - <<PY
import ruamel.yaml, os
fn = "$TMP_CONFIG"
y = ruamel.yaml.YAML()
with open(fn, 'r', encoding='utf-8') as f:
    cfg = y.load(f) or {}
cfg.setdefault('data',{})['start_date'] = "$TRAIN_START"
cfg['data']['end_date'] = "$TRAIN_END"
cfg['data']['test_start'] = "$TEST_START"
cfg['data']['test_end'] = "$TEST_END"
cfg.setdefault('training',{})['epochs'] = ${EPOCHS_FOR_QUICK_RUN}
with open(fn, 'w', encoding='utf-8') as f:
    y.dump(cfg, f)
print("已写入临时配置", fn)
PY

  echo "训练模型（将日志/检查点写入 Lightning logdir）..."
  # 用脚本支持的参数：--logdir 指向日志根目录（脚本会在 version_x 下写入）
  # 把输出日志保存到 RESULTS_DIR 以便排查
  "$PYTHON_BIN" scripts/training.py --config "$TMP_CONFIG" --logdir "$CHECKPOINTS_ROOT" --save_checkpoints --max_epochs ${EPOCHS_FOR_QUICK_RUN} 2>&1 | tee "$RESULTS_DIR/train_${TASK_IDX}.log" || true

  # 在 logs/CMamba 下查找最新的 .ckpt（跨 version 目录）
  LATEST_CKPT=$(find "$CHECKPOINTS_ROOT" -type f -name "*.ckpt" -print0 2>/dev/null | xargs -0 ls -t 2>/dev/null | head -n1 || true)
  if [ -z "$LATEST_CKPT" ]; then
    echo "未找到 checkpoint（在 $CHECKPOINTS_ROOT 下没有 .ckpt 文件）。请查看 $RESULTS_DIR/train_${TASK_IDX}.log 以排查训练保存路径。" >&2
    continue
  fi
  echo "找到最新 checkpoint: $LATEST_CKPT"

  echo "评估模型..."
  "$PYTHON_BIN" scripts/evaluation.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" > "$RESULTS_DIR/eval_${TASK_IDX}.log" || true

  echo "回测（test split）..."
  "$PYTHON_BIN" scripts/simulate_trade.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" --split test --trade_mode smart > "$RESULTS_DIR/backtest_${TASK_IDX}.log" || true

  echo "准备单日预测输入 one_day_input.csv..."
  "$PYTHON_BIN" - <<PY
import pandas as pd
df = pd.read_csv("data/raw.csv")
# 尝试兼容不同的日期列名
for col in ['Date','date','Timestamp','timestamp']:
    if col in df.columns:
        df['Date'] = pd.to_datetime(df[col], errors='coerce')
        break
else:
    df['Date'] = pd.to_datetime(df.iloc[:,0], errors='coerce')
end = pd.to_datetime("$TEST_END")
subset = df[df['Date'] <= end].tail(90)
subset.to_csv("one_day_input.csv", index=False)
print("one_day_input.csv 已生成", subset.shape)
PY

  echo "单日预测..."
  "$PYTHON_BIN" scripts/one_day_pred.py --config "$TMP_CONFIG" --ckpt_path "$LATEST_CKPT" --input_csv one_day_input.csv > "$RESULTS_DIR/one_day_pred_${TASK_IDX}.log" || true

  echo "任务 $TASK_IDX 完成，结果保存在 $RESULTS_DIR"
done

echo "全部任务完成。汇总评估日志在 $RESULTS_DIR"


bash: line 27: TASK: unbound variable


CalledProcessError: Command 'b'set -euo pipefail\ncd /content/CryptoMamba_run/repo\n\n# \xe5\xae\x89\xe8\xa3\x85\xe5\xbf\x85\xe8\xa6\x81\xe5\xba\x93\xe7\x94\xa8\xe4\xba\x8e yaml \xe4\xbf\xae\xe6\x94\xb9\xef\xbc\x88\xe8\x8b\xa5\xe5\xb7\xb2\xe5\xae\x89\xe8\xa3\x85\xe4\xbc\x9a\xe8\xb7\xb3\xe8\xbf\x87\xef\xbc\x89\npip install ruamel.yaml --quiet\n\nWORKDIR="/content/CryptoMamba_run"\n# \xe6\x8c\x87\xe5\x90\x91\xe4\xbb\x93\xe5\xba\x93\xe6\x97\xa5\xe5\xbf\x97\xe7\x9b\xae\xe5\xbd\x95\xef\xbc\x88\xe5\x90\xab version_x/checkpoints \xe5\xad\x90\xe7\x9b\xae\xe5\xbd\x95\xef\xbc\x89\nCHECKPOINTS_ROOT="$WORKDIR/repo/CryptoMamba/logs/CMamba"\nRESULTS_DIR="$WORKDIR/results"\nmkdir -p "$RESULTS_DIR"\n\nCONFIG="cmamba_v"\nPYTHON_BIN="python3"\nEPOCHS_FOR_QUICK_RUN=3\n\n# \xe6\xa3\x80\xe6\x9f\xa5 rolling_tasks.json\nTASKS_COUNT=$(python3 -c "import json,sys;print(len(json.load(open(\'rolling_tasks.json\'))))")\nif [ "$TASKS_COUNT" -eq 0 ]; then\n  echo "\xe4\xbb\xbb\xe5\x8a\xa1\xe6\x95\xb0\xe4\xb8\xba 0\

In [ ]:
# 挂载 Google Drive 并将结果目录复制到 Drive（如需）
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/CryptoMamba_results
!cp -r /content/CryptoMamba_run/results /content/drive/MyDrive/CryptoMamba_results/
print("结果已复制到 /content/drive/MyDrive/CryptoMamba_results")


In [ ]:
# 挂载 Google Drive 并将结果目录复制到 Drive（如需）
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/CryptoMamba_results
!cp -r /content/CryptoMamba_run/results /content/drive/MyDrive/CryptoMamba_results/
print("结果已复制到 /content/drive/MyDrive/CryptoMamba_results")
